In [1]:
from nltk import download
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
download('stopwords')
stop_words = stopwords.words('english')

import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acdong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def _preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

def _lemmatize(tags):
    """단어 리스트를 받아 품사를 구분하여 원형으로 바꿔줍니다."""
    result = list()
    n=WordNetLemmatizer()
    for word , tag in tags:
        if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP']:
            result.append(n.lemmatize(word,'v'))
        else:
            result.append(n.lemmatize(word))
    return result

In [3]:
originText = """
For their own benefit, companies have various ways of offering lower prices. One way of doing this is a
trade discount. It is offered to the shops or businesses that buy goods on a large scale and sell them. There is
also a quantity discount, which is offered to individuals who order quantities of a product. The company gives a
price break to these buyers because they help cut the costs of selling, storing, shipping, and billing. Finally, a
cash discount is a lower price offered to people who pay in cash.
"""

In [4]:
options = ["Types of Discount Pricing",
           "Ways to Improve Products",
           "How to Buy Products in Cash",
           "Locations of Discount Stores",
           "How to Start a Business"]

answer = 1 #1번

In [5]:
import gensim.models.keyedvectors as word2vec
path = './GoogleNews-vectors-negative300-SLIM.bin.gz'
w2vModel = word2vec.KeyedVectors.load_word2vec_format(path, binary=True)

In [6]:
TextTokens = _preprocess(originText)
textTags = nltk.pos_tag(TextTokens)

In [7]:
text_words = _lemmatize(textTags)

In [8]:
#정답 리스트를 단어별로 쪼개서 원형으로 바꿔줍니다.
resultChoice = list()
for choice in options:
    tempChoice = _lemmatize(nltk.pos_tag(_preprocess(choice)))
    resultChoice.append(tempChoice)

In [9]:
# 본문에 있는 단어들과 보기단어들의 wmd의 거리를 계산하여 distanceList에 추가합니다.
# tuple 형식으로 번호를 추가하고 거리가 가까운 순으로 정렬합니다.
distanceList = []
for i in range(len(options)):
    distance = w2vModel.wmdistance(text_words,resultChoice[i])
    distanceList.append((distance , i))

distanceList.sort(key=lambda x : x[0])

In [10]:
distanceList

[(1.0948977324125935, 2),
 (1.1234846038598156, 0),
 (1.138055826572937, 3),
 (1.1765813972018972, 1),
 (1.2263925364015407, 4)]

In [31]:
def question_solving(originText , options ,answer):
    """
    Type : orginText(str) , options(list) , answer(int)
    0. args 의 전처리 과정을 거칩니다.
    1. 선택보기들과 본문단어들의 거리값들을 모아둔 distanceList에서 가장 가까운거리의 인덱스를 정답이라고 판단합니다.
    2. picked 와 answer를 비교하여 정답이면 1을 리턴 아니면 0을 리턴합니다.
    """
    text_words = _lemmatize(nltk.pos_tag(_preprocess(originText)))
    resultChoice = list()
    for choice in options:
        tempChoice = _lemmatize(nltk.pos_tag(_preprocess(choice)))
        resultChoice.append(tempChoice)
    
    distanceList = list()
    for choiceNum in range(len(options)):
        distance = w2vModel.wmdistance(text_words,resultChoice[choiceNum])
        distanceList.append(distance)
    
    picked = distanceList.index(min(distanceList)) + 1
    
    print("정답은 {}번 입니다.".format(picked))
    if(answer == picked):
        print("정답을 맞췄습니다.")
        return 1
    elif(answer != picked):
        print("틀렸습니다. 실제정답은 {} 입니다".format(answer))
        return 0

### 실제 데이터에 사용하여 성능 확인하기

In [43]:
data = pd.read_excel('text_data.xlsx')
data = data.replace(np.NaN, 'None')

In [44]:
#정답이 영어인 행만 추출
import re
index_list = list()
reg=re.compile(r'[a-zA-Z]')

for raw in range(len(data)):
    matchObj=reg.match(data['#2'][raw])
    if matchObj:index_list.append(raw)
        
data = data.loc[index_list,:].reset_index(drop=True)

In [45]:
#question_soving 함수가 정답이면 1을 리턴 하기때문에 맞은갯수 / 문제갯수를 통해 정확도를 계산합니다.
accuracy = 0
columns = ['#1','#2',"#3",'#4','#5','answer']
for i in range(len(data['text'])):
    origin_text = data['text'][i]
    options = data[columns].loc[i].values[:5]
    answer = data[columns].loc[i].values[-1]
    accuracy += question_solving(origin_text,options , answer)
print("정확도 : {}".format((accuracy / len(data['text']))))
print("{} 문제중 {}개의 정답을 맞췄습니다.".format(len(data['text']), accuracy ))

정답은 2번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 2번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 3번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 4번 입니다.
정답을 맞췄습니다.
정답은 2번 입니다.
정답을 맞췄습니다.
정답은 4번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 4번 입니다.
틀렸습니다. 실제정답은 3 입니다
정답은 3번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 5번 입니다.
틀렸습니다. 실제정답은 2 입니다
정답은 5번 입니다.
틀렸습니다. 실제정답은 4 입니다
정답은 5번 입니다.
정답을 맞췄습니다.
정답은 4번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 5번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 5번 입니다.
정답을 맞췄습니다.
정답은 1번 입니다.
정답을 맞췄습니다.
정답은 2번 입니다.
틀렸습니다. 실제정답은 5 입니다
정답은 2번 입니다.
정답을 맞췄습니다.
정답은 2번 입니다.
틀렸습니다. 실제정답은 5 입니다
정답은 2번 입니다.
틀렸습니다. 실제정답은 3 입니다
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 5번 입니다.
정답을 맞췄습니다.
정답은 1번 입니다.
정답을 맞췄습니다.
정답은 5번 입니다.
정답을 맞췄습니다.
정답은 5번 입니다.
틀렸습니다. 실제정답은 3 입니다
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 2번 입니다.
정답을 맞췄습니다.
정답은 4번 입니다.
틀렸습니다. 실제정답은 3 입니다
정답은 4번 입니다.
정답을 맞췄습니다.
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 1번 입니다.
정답을 맞췄습니다.
정답은 1번 입니다.
정답을 맞췄습니다.
정답은 3번 입니다.
정답을 맞췄습니다.
정답은 4번 입니다.
틀렸습니다. 실제정답은 1 입니다
정답은 2번 입니다.
틀렸습니다. 실제

In [36]:
def question_solving_generous(originText , options ,answer):
    """
    question_solving_generous 함수는 몇번시도만에 정답을 맞췄는지를 체크합니다.
    """
    text_words = _lemmatize(nltk.pos_tag(_preprocess(originText)))
    resultChoice = list()
    for choice in options:
        tempChoice = _lemmatize(nltk.pos_tag(_preprocess(choice)))
        resultChoice.append(tempChoice)
    
    distanceList = list()
    for choiceNum in range(len(options)):
        distance = w2vModel.wmdistance(text_words,resultChoice[choiceNum])
        distanceList.append((distance , choiceNum))
    distanceList.sort(key=lambda x : x[0])
    
    picked = distanceList[0][1] + 1
    picked2 = distanceList[1][1] + 1
    picked3 = distanceList[2][1] + 1

    tryOne = 0; tryTwo = 0; tryThree = 0;
    if(answer == picked): tryOne = 1
    elif(answer == picked2): tryTwo = 1
    elif(answer == picked3): tryThree = 1

    return tryOne , tryTwo , tryThree
    

In [42]:
try1 = 0
try2 = 0
try3 = 0
columns = ['#1','#2',"#3",'#4','#5','answer']
for i in range(len(data['text'])):
    origin_text = data['text'][i]
    options = data[columns].loc[i].values[:5]
    answer = data[columns].loc[i].values[-1]
    tryOne , tryTwo , tryThree = question_solving_generous(origin_text,options , answer)
    try1 += tryOne
    try2 += tryOne + tryTwo
    try3 += tryOne + tryTwo + tryThree
    
avg_score = round(try1 / len(data['text']),2) * 100
gen_score = round(try2 / len(data['text']),2) * 100
m_gen_score = round(try3 / len(data['text']),2) * 100
print(f'정답률은 {avg_score}%입니다.')
print(f'너그러운 정답률은 {gen_score}%입니다.' )
print(f'더 너그러운 정답률은 {m_gen_score}%입니다.')

정답률은 56.99999999999999%입니다.
너그러운 정답률은 75.0%입니다.
더 너그러운 정답률은 89.0%입니다.
